### Interpreting Logs
- `cat`: The category of the event (e.g., operator, execution, memory allocation)
  - `"Session"`: Session-level activities (e.g., model loading, session initialization)
  - `"Node"`: Node-level activities (e.g., execution of individual operations in the model graph)
- `name`: Name of the operator or event
- `dur`: Duration of the event (in microseconds)
- `ts`: Timestamp of the event (relative to the start of profiling)
- `ph`: Event Phase; Indicates the type of profiling event ("X" typically represents a complete event)
- `args`: Provides additional details about the event, such as operator information, shapes, sizes, and execution provider

### Suggestions for Analysis
1. **Verify Session Lifecycle:**
   - Check if your program is creating more sessions than expected. Look at where and how `OrtSession` objects are created
   - If unintended sessions are created, ensure proper session re-use or cleanup
2. **Aggregate Logs if Necessary:**
   - If you want a unified view, you can combine these logs into a single JSON or CSV for easier comparison of execution times and bottlenecks
3. **Use Profiling Tools:**
   - Tools like `chrome://tracing` (in Google Chrome) can load these logs to provide a visual timeline of execution events

In [ ]:
import json, re, os

def readLog(log_name):
    # Open and read the JSON file
    with open(log_name, 'r') as file:
        data = json.load(file)

    # Display the contents of the JSON file
    #for i in data: print(i)
    return data

def getLog(log_prefix):
    log_files = dict()
    files = os.listdir()    # Get the list of files in the current directory

    # Iterate through the list of files
    for file in files:
        # Add log files with matching prefixes
        if file.startswith(log_prefix):     #if re.match(log_prefix, file):
            with open(file, 'r') as f:
                log_files[file] = json.load(f)

    return log_files

#data = readLog("gibb_profile.log_2024-12-05_18-21-56.json")
p = getLog("gibb_profile")


In [6]:
for k,v in p.items():
    print(f'({len(v)}) "{k}":')
    if k != "gibb_profile.log_2024-12-05_18-21-56.json":
        for event in v:
            #print("  >", i)
            ts = event['ts'] / 1_000_000    # Timestamp in seconds
            dur = event['dur'] / 1_000_000  # Duration in seconds
            # Remove some fields
            filtered_event = {key: value for key, value in event.items() if key not in ['pid', 'tid', 'cat', 'name', 'dur', 'ts']}
            print(f"  [{ts:.2f}] {event['cat']}: \"{event["name"]}\" Duration =  {dur:.3f} sec")#, filtered_event)

        print()
        continue
    
    session_events = [event for event in v if event['cat'] == 'Session']
    for event in session_events:
        ts = event['ts'] / 1_000_000    # Timestamp in seconds
        dur = event['dur'] / 1_000_000  # Duration in seconds
        ph = event['ph']
        filtered_event = {key: value for key, value in event.items() if key not in ['pid', 'tid', 'cat', 'name', 'dur', 'ts']}

        print(f"  [{ts:.2f}] {event['cat']}: \"{event["name"]}\"")#, filtered_event)
        print(f"    - Duration: {dur:.3f} sec")
        if ph != "X": print(f"    - Event Phase: {ph}")
        if event['args'] != {}: print(f"    - Args: {event['args']}")
        print()


(2) "gibb_profile.log_2024-12-05_18-21-53.json":
  [0.00] Session: "model_loading_uri" Duration =  0.296 sec
  [0.30] Session: "session_initialization" Duration =  0.486 sec

(2) "gibb_profile.log_2024-12-05_18-21-54.json":
  [0.00] Session: "model_loading_uri" Duration =  0.440 sec
  [0.44] Session: "session_initialization" Duration =  0.596 sec

(2) "gibb_profile.log_2024-12-05_18-21-55.json":
  [0.00] Session: "model_loading_uri" Duration =  0.409 sec
  [0.41] Session: "session_initialization" Duration =  0.557 sec

(778) "gibb_profile.log_2024-12-05_18-21-56.json":
  [0.00] Session: "model_loading_uri"
    - Duration: 0.184 sec

  [0.18] Session: "session_initialization"
    - Duration: 0.236 sec

  [0.54] Session: "SequentialExecutor::Execute"
    - Duration: 0.042 sec

  [0.54] Session: "model_run"
    - Duration: 0.042 sec



In [20]:
data = readLog("gibb_profile.log_2024-12-05_18-21-56.json")
g = ["/distilbert/embeddings/", "/distilbert/transformer/layer"]

data = [el for el in data if el['cat'] == 'Node']
print(len(data))
for i in g:
    d = [el for el in data if el['name'].startswith(i)]
    data = [el for el in data if not el['name'].startswith(i)]
    total_dur = sum(el['dur'] for el in d)
    print(f"Total duration for '{i}': {total_dur}")

print(len(data))

for i in data: print(f"Node - '{i['name']}':", i['dur'])

774
Total duration for '/distilbert/embeddings/': 169
Total duration for '/distilbert/transformer/layer': 38349
27
Node - 'Gelu_fence_before': 0
Node - 'Gelu_kernel_time': 247
Node - 'Gelu_fence_after': 0
Node - 'Gelu_token_0_fence_before': 0
Node - 'Gelu_token_0_kernel_time': 231
Node - 'Gelu_token_0_fence_after': 0
Node - 'Gelu_token_1_fence_before': 0
Node - 'Gelu_token_1_kernel_time': 227
Node - 'Gelu_token_1_fence_after': 0
Node - 'Gelu_token_2_fence_before': 0
Node - 'Gelu_token_2_kernel_time': 226
Node - 'Gelu_token_2_fence_after': 0
Node - 'Gelu_token_3_fence_before': 0
Node - 'Gelu_token_3_kernel_time': 225
Node - 'Gelu_token_3_fence_after': 0
Node - 'Gelu_token_4_fence_before': 0
Node - 'Gelu_token_4_kernel_time': 225
Node - 'Gelu_token_4_fence_after': 0
Node - '/Gather_fence_before': 0
Node - '/Gather_kernel_time': 4
Node - '/Gather_fence_after': 0
Node - 'fused /pre_classifier/Gemm_fence_before': 0
Node - 'fused /pre_classifier/Gemm_kernel_time': 194
Node - 'fused /pre_clas

In [26]:
len([    "Load Config", 
    "Create Geco", 
    "GEC Preproc", 
    "Encoder", 
    "Main Decoder", 
    "Past Decoder", 
    "All Decoders", 
    "Get Max Tokens", 
    "GEC Postproc",
    "Grammar", 
    "Gibb Preproc",
    "Calculate Softmax",
    "Gibberish",
    "Destroy Geco", ])

14

In [23]:
x = """
#include <assert.h>
#include <json-c/json.h>
#include <math.h>
#include <stdbool.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <stdarg.h>
#include <string.h>
"""
# Sort the lines of the string alphabetically
sorted_lines = sorted(x.strip().split('\n'))

# Join the sorted lines back into a single string
x_sorted = '\n'.join(sorted_lines)

print(x_sorted)

#include <assert.h>
#include <json-c/json.h>
#include <math.h>
#include <stdarg.h>
#include <stdbool.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <string.h>
#include <time.h>


In [4]:
import os

# List the file names in this directory
# List the file names in the current directory
file_names = os.listdir('.')
for i in file_names:
    if i.startswith("prof2"): print(f'"{i}"')

"prof2.log_2024-12-05_17-48-23.json"
"prof2.log_2024-12-05_17-48-24.json"
"prof2.log_2024-12-05_17-48-25.json"
"prof2.log_2024-12-05_17-48-26.json"


In [9]:
from collections import Counter

def getFieldCounts(data, fieldName):
    # Count the occurrences of each unique value in the "cat" field
    val_counts = Counter(item[fieldName] for item in data)
    
    # Display the counts
    print(val_counts)

getFieldCounts(data, "cat")
getFieldCounts(data, "ph")


Counter({'Node': 2100, 'Session': 4})
Counter({'X': 2104})


In [7]:
x = 436292
# Convert the value from microseconds to milliseconds
x_ms = x / 1000
print(x_ms)


436.292


In [12]:
# Sort the data by the "dur" field in descending order and get the first 10 objects
top_10_dur = sorted(data, key=lambda x: x['dur'], reverse=True)[:10]

# Display the top 10 objects
for item in top_10_dur:
    print("ITEM:", item)
    print(f" > Category: {repr(item['cat'])}")
    print(f" > Duration: {round((item['dur']/1000), 2)} ms")
    #print(f" > Category: {repr(item['cat'])}")
    print()


ITEM: {'cat': 'Session', 'pid': 75084, 'tid': 75084, 'dur': 436292, 'ts': 430001, 'ph': 'X', 'name': 'session_initialization', 'args': {}}
 > Category: 'Session'
 > Duration: 436.29 ms

ITEM: {'cat': 'Session', 'pid': 75084, 'tid': 75084, 'dur': 429995, 'ts': 1, 'ph': 'X', 'name': 'model_loading_uri', 'args': {}}
 > Category: 'Session'
 > Duration: 430.0 ms

ITEM: {'cat': 'Session', 'pid': 75084, 'tid': 75084, 'dur': 59052, 'ts': 1840253, 'ph': 'X', 'name': 'model_run', 'args': {}}
 > Category: 'Session'
 > Duration: 59.05 ms

ITEM: {'cat': 'Session', 'pid': 75084, 'tid': 75084, 'dur': 58938, 'ts': 1840309, 'ph': 'X', 'name': 'SequentialExecutor::Execute', 'args': {}}
 > Category: 'Session'
 > Duration: 58.94 ms

ITEM: {'cat': 'Node', 'pid': 75084, 'tid': 75084, 'dur': 7585, 'ts': 1891084, 'ph': 'X', 'name': '/lm_head/MatMul_kernel_time', 'args': {'thread_scheduling_stats': '', 'output_type_shape': [{'float': [5, 1, 32128]}], 'output_size': '642560', 'parameter_size': '0', 'activation_

In [2]:
# Filter out objects where the "cat" field is "Node"
filtered_data = [item for item in data if item['cat'] != 'Node']

# Display the length of the filtered data
print(len(filtered_data))


4
